based on Matt Harrison's "Effective Pandas"

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

## Exploratory data analysis

In [ ]:
df = pd.read_csv("../harrison_data/data/vehicles.csv.zip")
# df.info()
df.describe()

In [ ]:
# df.head()
df.sample(5, random_state=42)

In [ ]:
df.make.value_counts()

## dtype conversion and memory usage

In [ ]:
df.barrels08.astype('float16').nbytes / 1024

In [ ]:
df.barrels08.astype('float64').nbytes / 1024

In [ ]:
df.barrels08.memory_usage() / 1024

In [ ]:
df.barrels08.memory_usage(deep=True) / 1024

In [ ]:
df.city08.memory_usage(deep=True) / 1024

In [ ]:
df.city08.astype('category').memory_usage()/ 1024

In [ ]:
type(df.city08.to_frame())

## If ... then ... else ... 

- .apply(func)
- .where(true_array, other)
- np.select([true_array1, ...], [replacement1, ...], default)

In [ ]:
top5_as_index = df.make.value_counts().index[:5]
top5_series = pd.Series(df.make.value_counts().index[:5])

In [ ]:
make = df.make

In [ ]:
def set_outside_top_5_to_other(ser_elem):
    if ser_elem in df.make.value_counts().index[:5]:
        return ser_elem
    else:
        return 'Other'
    

In [ ]:
make.apply(set_outside_top_5_to_other)

In [ ]:
make.where(make.isin(top5_series), other='Other')

In [ ]:
make.mask(~make.isin(top5_series), other='Other')

In [ ]:
top10 = df.make.value_counts().index[:10]
top5 = df.make.value_counts().index[:5]

In [ ]:
np.select([make.isin(top5), make.isin(top10)], [make, "top 10"], "other")

## Missing data

- .isna()
- .fillna()
- .interpolate()
- .clip(lower = , upper= )
- .drop_duplicates()

In [ ]:
cyl = df.cylinders
cyl.head()

In [ ]:
missing = cyl.isna()
make.loc[missing]

In [ ]:
cyl.fillna(0).loc[missing]

In [ ]:
temp = pd.Series([1, 2, 3, 4, 5, 6, np.nan, 8, 9, 10])
temp

In [ ]:
temp.interpolate()

In [ ]:
(df.barrels08
 .clip(lower = df.barrels08.quantile(0.05),
       upper = df.barrels08.quantile(0.95))
)

In [ ]:
df.barrels08.drop_duplicates()

In [ ]:
make.replace(
    {'Subaru': "Sulama", 
     "Volvo": "Ovlov"}
    ).loc[make == 'Subaru'] # make == 'Volvo']

In [ ]:
make.replace(r'(Fer)ra(r.*)', r'\2-other-\1', regex=True) 

## Sort, rank

In [ ]:
df.barrels08.head()

In [ ]:
df.barrels08.sort_values()

In [ ]:
df.barrels08.sort_values().sort_index()

In [ ]:
df.barrels08.rank()

In [ ]:
pd.cut(df.barrels08, 10) # each bin has 10 elements

In [ ]:
pd.qcut(df.barrels08, 10) # set number of bins = 10 
# pd.qcut(df.barrels08, 10, labels = list('ABCDEFGHIJ'))

# Indexing

In [ ]:
city08 = df.city08
highway08 = df.highway08

In [ ]:
city08.head()

In [ ]:
city08

In [ ]:
city2 = city08.rename(index = make.to_dict())
city2

In [ ]:
city2.reset_index()

In [ ]:
city2.sort_index().loc["Ferarri":"Lamborghini"]

In [ ]:
city2.sort_index().loc["F":"L"] # includes anything that starts with F until precisely a CHAR L, exclude strings staring with L

In [ ]:
# .iloc indexing with boolean Series is NOT supported
city2.iloc[(make == 'Subaru').to_numpy()]

In [ ]:
type(make == "Subaru")

In [ ]:
city2.filter(regex="(Ford)|(Subaru)")

In [ ]:
s1 = pd.Series([1, 2, 3, 4, 5], index = list('abcde'))
s2 = pd.Series([10, 20, 30, 40, 50], index = list('cdefg'))
s2.reindex(s1.index)

# TimeSeries

In [ ]:
# dirty devil river data
df = pd.read_csv("../harrison_data/data/dirtydevil.txt", skiprows = lambda num: num < 34 or num == 35, sep = '\t')
df.head()

In [ ]:
def tweak_rivers(df):
    return (df
            .assign(datetime = pd.to_datetime(df.datetime))
            .rename(columns = {
                '144166_00060': 'cfs',
                '144167_00065': 'gage_height'
            })
            .set_index('datetime')
            )

In [ ]:
dd = tweak_rivers(df)

In [ ]:
dd

In [ ]:
# adding timezone information
def to_america_denver_time(df_, time_col: str, tz_col: str):
    return (df_
            .assign(**{tz_col: df_[tz_col].replace('MDT', 'MST7MDT' )})
            .groupby(tz_col)
            [time_col]
            .transform(lambda s: pd.to_datetime(s)
                    .dt.tz_localize(s.name, ambiguous = True)
                    .dt.tz_convert('America/Denver'))
            )

def tweak_rivers(df):
    return (df
            .assign(datetime = to_america_denver_time(df, 'datetime', 'tz_cd'))
            .rename(columns = {
                '144166_00060': 'cfs',
                '144167_00065': 'gage_height'
            })
            .set_index('datetime')
            )

In [ ]:
dd = tweak_rivers(df)

In [ ]:
dd.head()

In [ ]:
dd.index[0]

In [ ]:
fig, ax = plt.subplots(dpi=600)
dd.cfs.plot()

In [ ]:
dd.cfs.describe()

In [ ]:
# dd18 = dd.cfs.loc['2018/3':].clip(upper=400)
dd18 = dd.cfs.sort_index().loc['2018/3':'2019/5'].clip(upper=400)

In [ ]:
ax = dd18.plot()
dd18.resample('D').mean().plot(ax=ax, label='daily')
dd18.resample('D').mean().rolling(7).mean().plot(ax=ax, label='weekly')
ax.legend()

In [ ]:
dd.index[0:3]

In [ ]:
dd.cfs